In [20]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.prompts import PromptTemplate
import json
import requests
from pydantic import BaseModel

api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERP_API_KEY")
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "Claude"
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

In [1]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [6]:
chat = ChatAnthropic()

In [11]:
messages = [
    HumanMessage(
        content="write a 150 words on python"
    )
]
chat(messages)

 Here is a 149 word essay on Python:

Python is a popular high-level programming language that was created by Guido van Rossum in the early 1990s. It has simple, easy-to-use syntax that emphasizes readability and therefore reduces the cost of program maintenance. Python supports modules and packages, which encourages program modularity and code reuse. The Python interpreter and extensive standard library are available in source or binary form for all major platforms, and can be freely distributed. Python has a large and comprehensive standard library which includes modules for many common programming tasks such as connecting to web servers, searching text with regular expressions, reading and modifying files. Python's interactive mode makes it easy to experiment with features and try out code fragments. Python can also be embedded in existing applications to provide a programmable interface. Python's high-level data types allow developers to express concepts without worrying about low-

AIMessage(content=" Here is a 149 word essay on Python:\n\nPython is a popular high-level programming language that was created by Guido van Rossum in the early 1990s. It has simple, easy-to-use syntax that emphasizes readability and therefore reduces the cost of program maintenance. Python supports modules and packages, which encourages program modularity and code reuse. The Python interpreter and extensive standard library are available in source or binary form for all major platforms, and can be freely distributed. Python has a large and comprehensive standard library which includes modules for many common programming tasks such as connecting to web servers, searching text with regular expressions, reading and modifying files. Python's interactive mode makes it easy to experiment with features and try out code fragments. Python can also be embedded in existing applications to provide a programmable interface. Python's high-level data types allow developers to express concepts withou

In [8]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [9]:
await chat.agenerate([messages])

LLMResult(generations=[[ChatGeneration(text=' Voici la traduction en français de la phrase "I love programming" :\n\nJ\'aime la programmation.', generation_info=None, message=AIMessage(content=' Voici la traduction en français de la phrase "I love programming" :\n\nJ\'aime la programmation.', additional_kwargs={}, example=False))]], llm_output={}, run=[RunInfo(run_id=UUID('a302045d-0015-4e62-991b-e6ef19407c45'))])

In [10]:
chat = ChatAnthropic(
    streaming=True,
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)
chat(messages)

 Voici la traduction en français : J'adore programmer.

AIMessage(content=" Voici la traduction en français : J'adore programmer.", additional_kwargs={}, example=False)

In [21]:
from langchain.document_loaders import WebBaseLoader


loader = WebBaseLoader('https://github.com/poojanvig')


github_profile = loader.load()

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSanchay-T (Sanchay) · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n\n            Sign\xa0up\n          \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nPackages\n        Host and manage packages\n      \n\n\n\n\n\n\n\nSecurity\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev environments\n      \n\n\n\n\n\n\n\nCopilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nCode review\n        Manage code changes\n      \n\n\n\n\n\n\n\nIssues\n        Plan and track work\n      \n\n\n\n\n\n\n\nDiscussions\n        Collaborate outside of code\n      \n\n\n\n\nExplore\n\n\n\n      All features\n\n    

In [13]:
messages = [
    HumanMessage(
        content=f'You are an expert Github Analyst and experiece in leading top projects and an awesome project manager.Your job is to analyse this github profile : {docs} and extract key contents in json and summarize the most important things in great detail.'
    )
]
chat(messages)

 Here is a detailed summary of the key information I extracted from the GitHub profile of Sanchay-T:

```json
{
  "basic_info": {
    "name": "Sanchay-T", 
    "url": "https://github.com/Sanchay-T",
    "description": "Sanchay-T has 38 repositories available. Follow their code on GitHub.",
    "location": null,
    "company": null,
    "followers": 2,
    "following": 6
  },
  "activity": {
    "contributions_last_year": 466,
    "contributions_by_day": {
      "busiest_day": "Saturday, November 12, 2022 (87 contributions)",
      "typical_days": "0-5 contributions", 
      "no_contributions": "113 days"  
    },
    "repositories_created": {
      "total": 38,
      "last_3_months": 3,
      "popular_languages": ["Python", "Java", "CSS"]
    },
    "pull_requests": {  
      "opened_last_3_months": 1
    },
    "stars": {
      "total": 46,
      "g

AIMessage(content=' Here is a detailed summary of the key information I extracted from the GitHub profile of Sanchay-T:\n\n```json\n{\n  "basic_info": {\n    "name": "Sanchay-T", \n    "url": "https://github.com/Sanchay-T",\n    "description": "Sanchay-T has 38 repositories available. Follow their code on GitHub.",\n    "location": null,\n    "company": null,\n    "followers": 2,\n    "following": 6\n  },\n  "activity": {\n    "contributions_last_year": 466,\n    "contributions_by_day": {\n      "busiest_day": "Saturday, November 12, 2022 (87 contributions)",\n      "typical_days": "0-5 contributions", \n      "no_contributions": "113 days"  \n    },\n    "repositories_created": {\n      "total": 38,\n      "last_3_months": 3,\n      "popular_languages": ["Python", "Java", "CSS"]\n    },\n    "pull_requests": {  \n      "opened_last_3_months": 1\n    },\n    "stars": {\n      "total": 46,\n      "g', additional_kwargs={}, example=False)

In [18]:
messages = [
    HumanMessage(
        content=f'You are an expert Github Analyst and experiece in leading top projects and an awesome project manager.Your job is to analyse this github profile : {docs} and extract key contents in json and summarize the most important things in great detail.'
    )
]
chat(messages)

 Here is a summary of the key information extracted from the GitHub profile of poojanvig in JSON format:

```json
{
  "name": "Poojan Vig", 
  "username": "poojanvig",
  "url": "https://github.com/poojanvig",
  "bio": "I am a highly motivated engineer with a passion for artificial intelligence and data science. I have a strong foundation in math, statistics, and computer science, and am proficient in programming languages such as Python and R. My experience includes working with machine learning algorithms, collecting, cleaning and analyzing data, as well as building and evaluating predictive models. I stay current with the latest advancements in the field by reading research papers and attending conferences and workshops. I am seeking opportunities to apply my skills to real-world problems and continue to develop my knowledge and experience in the field of AI and data science.",
  "location": null,
  "followers": 4,
  "following": 6,
  "public_repos": 6,
  "stars": 0,
  "contributions

AIMessage(content=' Here is a summary of the key information extracted from the GitHub profile of poojanvig in JSON format:\n\n```json\n{\n  "name": "Poojan Vig", \n  "username": "poojanvig",\n  "url": "https://github.com/poojanvig",\n  "bio": "I am a highly motivated engineer with a passion for artificial intelligence and data science. I have a strong foundation in math, statistics, and computer science, and am proficient in programming languages such as Python and R. My experience includes working with machine learning algorithms, collecting, cleaning and analyzing data, as well as building and evaluating predictive models. I stay current with the latest advancements in the field by reading research papers and attending conferences and workshops. I am seeking opportunities to apply my skills to real-world problems and continue to develop my knowledge and experience in the field of AI and data science.",\n  "location": null,\n  "followers": 4,\n  "following": 6,\n  "public_repos": 6,\

In [24]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
# Define your desired data structure.
from typing import Optional, Dict
from pydantic import BaseModel, Field, validator

class ContributionsByDay(BaseModel):
    busiest_day: str
    typical_days: str
    no_contributions: str

class RepositoriesCreated(BaseModel):
    total: int
    last_3_months: int
    popular_languages: list[str]

class PullRequests(BaseModel):
    opened_last_3_months: int

class Stars(BaseModel):
    total: int

class Activity(BaseModel):
    contributions_last_year: int
    contributions_by_day: ContributionsByDay
    repositories_created: RepositoriesCreated
    pull_requests: PullRequests
    stars: Stars

class BasicInfo(BaseModel):
    name: str
    url: str
    description: str
    location: Optional[str]
    company: Optional[str]
    followers: int
    following: int

class GithubAgent(BaseModel):
    basic_info: BasicInfo
    activity: Activity

    @validator("basic_info")
    def validate_basic_info(cls, basic_info):
        if not basic_info.name or not basic_info.url:
            raise ValueError("Name and URL are required!")
        return basic_info



# And a query intented to prompt a language model to populate the data structure.
model = ChatAnthropic(model="claude-2", temperature=0)


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=GithubAgent)

prompt = PromptTemplate(
    template="You are an expert Github Analyst and experiece in leading top projects and an awesome project manager.Your job is to analyse this github profile : {github_profile} and extract key contents in json with \n{format_instructions}\n and summarize the most important things in great detail",
    input_variables=["github_profile"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=prompt)

output = model(_input.to_string())

parser.parse(output)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. ' format_instructions' (type=value_error)